In [ ]:
import os
import tables
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq

In [ ]:
data_dir = '../data/IEEE39/converted_from_PowerFactory/all_stoch_loads/var_H_area_1_comp_grid'
H = [5.0, 4.33, 4.47, 3.57, 4.33, 4.35, 3.77, 3.47, 3.45, 4.2, 0.1, 0.1, 0.1]
P_frac = [None, 0.1, 0.3, 0.5, 0.7, 0.8]
data_files = []
for P in P_frac:
    fname = 'ieee39_PF_stoch_loads_compensators_'
    if P is not None:
        fname += 'vsg_Pfrac={:.1f}_'.format(P)
    fname += '_'.join(list(map(lambda s: f'{s:.03f}', H))) + '.h5'
    data_files.append(fname)
N_files = len(data_files)

In [ ]:
fids = [tables.open_file(os.path.join(data_dir, f)) for f in data_files]
buses = 3, 14, 17, 39
t = [fid.root.time.read() for fid in fids]
Vd = {bus: [fid.root[f'Vd_bus{bus}'].read() for fid in fids] for bus in buses}
for fid in fids:
    fid.close()
for bus in buses:
    for i in range(1,N_files):
        Vd[bus][i][-1] = Vd[bus][i][:-1].mean()

In [ ]:
dt = np.diff(t[0][:2])[0]
dur = 60
N_samples = int(60 / dt)
N_trials = t[0].size // N_samples
Xf = {}
for bus in buses:
    Xf[bus] = []
    for i in range(N_files):
        X = np.reshape(Vd[bus][i][:N_samples*N_trials], (N_trials,-1))
        tmp = fft(X)
        Xf[bus].append(2.0 / N_samples * np.abs(tmp[:, :N_samples//2]))
F = fftfreq(N_samples, dt)[:N_samples//2]

In [ ]:
cmap = plt.get_cmap('viridis', N_files)
bus = 3
fig,ax = plt.subplots(2, 1)
lw = 1
for i,P in enumerate(P_frac):
    if P is None:
        ax[0].plot(t[i]/60, Vd[bus][i]*1e-3, color='k', lw=lw/2)
        ax[1].semilogx(F, 20*np.log10(Xf[bus][i].mean(axis=0)), 'k', lw=lw, label='No VSG')
    else:
        ax[0].plot(t[i]/60, Vd[bus][i]*1e-3, color=cmap(i), lw=lw/2)
        ax[1].semilogx(F, 20*np.log10(Xf[bus][i].mean(axis=0)), color=cmap(i), lw=lw, label=f'P_frac={P}')
ax[0].set_xlabel('Time [min]')
ax[0].set_ylabel(f'Vd,{bus} [kV]')
ax[1].set_xlabel('Frequency [Hz]')
ax[1].set_ylabel(f'PSD [dB]')
ax[1].set_xlim([0.2, 20])
ax[1].set_ylim([-20,30])
ax[1].legend(loc='upper right', frameon=False, fontsize=8)
for a in ax:
    a.grid(which='major', axis='y', lw=0.5, color=[.6,.6,.6], ls=':')
    for side in 'right','top':
        a.spines[side].set_visible(False)
fig.tight_layout()